In [ ]:
#The original longer code that takes about an hour to run
#Isn't commented just here for reference

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pytickersymbols import PyTickerSymbols

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
#This Cell Block is all the functions for the strat. This is the only cell block that needs to be changed to backtest a different strategy.
def RSI(data, window=14):
    #daily_data = data.resample('D').last()
    
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

def strat(ticker, period, buy, sell):   
    # Download stock data from Yahoo Finance
    data = yf.download(ticker, period = period, interval = "1d")
    
    # Calculate RSI and add it to the DataFrame
    data['RSI'] = RSI(data)
    
    # Define the buy and sell conditions based on RSI
    data['Signal'] = 0
    data['Signal'][data['RSI'] > buy] = 1   # Buy signal
    data['Signal'][data['RSI'] > sell] = -1  # Sell signal
    
    # Shift signals so we enter positions on the next day
    data['Position'] = data['Signal'].shift(1)
    
    
    # Calculate daily returns
    data['Daily Return'] = data['Close'].pct_change()
    # Calculate strategy returns: Position * Daily Return
    data[ticker] = data['Position'] * data['Daily Return']
    
    return data[ticker]

In [3]:
daily = pd.DataFrame({
        'Date': []
    })
daily.set_index('Date', inplace=True)


In [4]:
def calculate_portfolio_sharpe_ratio(daily, risk_free_rate=0.011059):
    daily = daily.drop(daily.index[0])
    if daily.isnull().values.any():
        print(f"Warning: Missing data for some stocks in the portfolio.")
        null_counts = daily.isnull().sum()  # Get the number of nulls per column
        non_zero_nulls = null_counts[null_counts > 0]  # Filter out the columns where null count is 0
        print(non_zero_nulls)  # Print only columns with null values
        daily = daily.dropna(axis=1, how='any')  # Drop stocks with missing data

    print(daily)
    num = daily.shape[1]
    print(num)

    weight = [1 / num] * num

    weights = np.array(weight)
    
    portfolio_daily_returns = daily.dot(weights)

    # Calculate annualized return for the portfolio
    annualized_return = ((1 + portfolio_daily_returns.mean()) ** 64) - 1

    # Calculate annualized volatility (standard deviation) for the portfolio
    annualized_volatility = portfolio_daily_returns.std() * np.sqrt(64)
    
    # Sharpe ratio formula: (Expected Return - Risk-Free Rate) / Volatility
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility
    
    return sharpe_ratio

In [5]:
def portfolio_daily_returns(daily, symbol, buy, sell):
    daily = pd.concat([daily, strat(symbol, "3mo", buy, sell)], axis=1)

    return daily

In [8]:
parameters = pd.DataFrame({'Parameters':[], 'Sharpe':[]})
parameters.set_index('Parameters', inplace=True)


In [ ]:
stock_data = PyTickerSymbols()
sp500_symbols = stock_data.get_stocks_by_index('S&P 500')
symbols = [stock['symbol'] for stock in sp500_symbols]

sell = 70
for buy in range(50,101):
    daily = pd.DataFrame({'Date': []})
    daily.set_index('Date', inplace=True)

    for symbol in symbols:
        try:
            daily = portfolio_daily_returns(daily, symbol, buy, sell)
        except IndexError:
            pass

    sharpe = calculate_portfolio_sharpe_ratio(daily)

    
    pars = str(buy) + "," + str(sell)
    new_row = pd.DataFrame({'Sharpe': [sharpe]}, index=[pars])
    parameters = pd.concat([parameters, new_row])
    sell += 1

In [ ]:
parameters

In [11]:
parameters.to_csv('Parameter RSI sharpe test.csv', index=True)